# **Dataset & Fraud Detection**

## Objectives

* The objectives are to use AI tools to balance the dataset, generate data on where it is taking place, and produce visualisations.

## Inputs

* I will load in the ecommerce_transactions_cleaned dataset.

## Outputs

* A balanced dataset, data, and visualisations.

## Additional Comments

* Copilot will be used for this task to see how AI completes this task.



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\rayaf\\OneDrive\\Documents\\global-store\\online_store\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\rayaf\\OneDrive\\Documents\\global-store\\online_store'

In [4]:
import pandas as pd

# Stage 1

#### Loading the Dataset

In [5]:
fraud_df = pd.read_csv('../online_store/data/balanced_data/ecommerce_transactions_cleaned.csv')
fraud_df.head()

,Transaction_ID,User_Name,Age,Country,Product_Category,Purchase_Amount,Payment_Method,Transaction_Date,Year,Month,Day,Product_Category_Code,Segment
0,33554,Isabella Lewis,24,Japan,Toys,579.51,Cash on Delivery,2024-01-16,2024,1,16,7,1
1,9428,Elijah Rodriguez,52,Germany,Electronics,78.18,PayPal,2023-04-19,2023,4,19,3,2
2,200,Ava Hall,62,UK,Toys,713.08,Debit Card,2024-03-05,2024,3,5,7,0
3,12448,Ava Allen,63,Brazil,Grocery,474.14,Credit Card,2024-12-01,2024,12,1,4,2
4,39490,Emma Lewis,52,USA,Home & Kitchen,266.15,Debit Card,2024-01-19,2024,1,19,5,2


# Stage 2

#### Balancing the Dataset

In [6]:
from sklearn.utils import resample

# Check class distribution before balancing
print(fraud_df['Segment'].value_counts())

# Perform random oversampling to balance the classes

# Separate majority and minority classes
majority_class = fraud_df['Segment'].value_counts().idxmax()
dfs = []
max_count = fraud_df['Segment'].value_counts().max()

for segment in fraud_df['Segment'].unique():
    df_segment = fraud_df[fraud_df['Segment'] == segment]
    if len(df_segment) < max_count:
        df_segment_balanced = resample(df_segment, 
                                       replace=True, 
                                       n_samples=max_count, 
                                       random_state=42)
    else:
        df_segment_balanced = df_segment
    dfs.append(df_segment_balanced)

balanced_fraud_df = pd.concat(dfs).sample(frac=1, random_state=42).reset_index(drop=True)

# Check class distribution after balancing
print(balanced_fraud_df['Segment'].value_counts())

Segment
3    2612
1    2498
2    2469
0    2421
Name: count, dtype: int64
Segment
0    2612
1    2612
3    2612
2    2612
Name: count, dtype: int64


# Stage 3

Detecting Fraud 

# Stage 4

#### Fraud Visualisations